In [9]:
import qbraid_algorithms
from qbraid_algorithms.qtran import QasmBuilder, std_gates, GateLibrary, GateBuilder

In [ ]:
qubits = range(3)

builder = QasmBuilder(3)

b=0

std = builder.import_library(lib_class=std_gates)
    
name = f"qaoa_cost_{len(qubits)}"  # BUG FIX: Include depth in name

qubit_list = "{" + ",".join([str(q) for q in qubits]) + "}"

qubit_array_param = f"qubit[{len(qubits)}] qubits"

std.begin_subroutine(
    name, [qubit_array_param]
)

for i in qubits:
    std.z(i)

std.end_subroutine()

In [28]:
print(name)

qaoa_cost_3


In [29]:
print(qubit_list)

{0,1,2}


In [30]:
qubit_array_param

'qubit[3] qubits'

In [31]:
program = builder.build()
print(program)

Warning (QasmBuilder): built qasm has unclosed scope, string will fail compile in native
OPENQASM 3;
include "stdgates.inc";
qubit[3] qb;
bit[3] cb;
def qaoa_cost_3(qubit[3] qubits) {
	rz(0.45) qb[0];
	rz(0.45) qb[1];
	rz(0.45) qb[2];

